In [49]:
#TODO:  after testing with sulfolobus, uncomment GTG in keep only valid orfs
import subprocess

def run_pipeline(file_prefix, input_path, output_path): 
    
    # pre-process to get rid of bad orfs
    cds_file = input_path + file_prefix + "_cds_from_genomic.fna.gz"
    processed_cds_file = output_path + file_prefix + "_cds_processed.fa"
    try:
        ret = subprocess.check_output(["perl", "keep_only_valid_orfs.pl", cds_file, processed_cds_file], stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError as exc:
        print("Problem executing keep_only_valid_orfs for", cds_file, "; ", exc.returncode, exc.output)
    
    # get the codon pair scores
    codon_pair_scores_file = output_path + file_prefix + "_pairwise_syn_shuffle_frm123.csv"
    codon_pair_scores_helper_file = output_path + file_prefix + "_obs_exp_frm123.cts"
    try:
        ret = subprocess.check_output(["perl", "alisa_efficient_codon_pair_score_hexamers_paper.pl", "-s", \
                                       processed_cds_file, "-o", file_prefix, "-z", "10"], stderr=subprocess.STDOUT)
        print(ret)
    except subprocess.CalledProcessError as exc:
        print("Problem executing alisa_efficient_codon_pair_score_hexamers for", cds_file, "; ", exc.returncode, exc.output)
    
    
    # cleanup: remove created temporary files
    #TODO: rm processed_cds_file, codon_pair_scores_file, codon_pair_scores_helper_file
    

input_file_path = "./"
output_file_path = "./"
file_prefix = "Sulfolobus_acidocaldarius"
run_pipeline(file_prefix, input_file_path, output_file_path)

b'hello\n'
